In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Korean", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Korean", task="transcribe")

/home/yumin/miniconda3/envs/whisper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yumin/miniconda3/envs/whisper/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from datasets import load_dataset
dataset = load_dataset("Bingsu/zeroth-korean")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 457
    })
})

In [4]:
dataset['test']['text']

['지난해 삼 월 김 전 장관의 동료인 장동련 홍익대 교수가 민간 자문단장으로 위촉되면서 본격적인 공모와 개발 작업에 들어갔다',
 '그 바람에 나의 몸도 겹쳐 쓰러지며 한창 피어난 노란 동백꽃 속으로 폭 파묻혀 버렸다',
 '현재 백화점과 영화관 등은 오픈해 영업하고 있고 테마파크 및 아파트 등의 공사는 이천 십 팔 년 완공을 목표로 진행돼 왔다',
 '또한 박 전 대통령 측은 앞으로 검찰 수사와 관련해 수사 내용에 대한 입장뿐 아니라 변호인단의 활동 상황 등도 소상하게 공개한다는 방침이다',
 '한 잔의 커피가 소비자에게 전해지기 위해선 여러 차례의 공정을 거쳐야 한다',
 '다른 지역에서는 그곳이 아무리 넓다 하더라도 푸야 라이몬디를 발견하기가 어려울 것입니다',
 '물론 십 대 자녀가 잘못된 결론을 내리거나 심지어 당신과 비슷한 실수를 저질렀을 때 자신을 정당화하지는 않을까 염려가 될 수도 있습니다',
 '만약 박근혜 대통령이 정말로 손을 내밀면 안철수 대표와 국민의당은 어떤 선택을 할까요',
 '대부분 전기는 기업이 사용 가정용 전기에 누진제 적용하는 국가 거의 없어 많이 쓴다고 돈 더 내게 하는 상품이 어딨나',
 '더 선은 이름이 밝혀지지 않은 이 노인이 이 년 전 피부암을 진단받은 뒤 자신의 양심과 씨름해왔다고 전했다',
 '생명력이 강한 이 미생물들은 바닷물에서 추출한 물질과 끈끈한 점액을 섞어서 접합제를 만든 다음 바위같이 생긴 집에 층층이 바릅니다',
 '때로는 바람이 풀밭을 스치는 소리를 들으면서 그 소리와는 대조적으로 크고도 찢어지는 듯이 날카로운 쌍띠물떼새의 울음소리가 물통 근처에서 들려오는 것에 귀를 기울이곤 하였습니다',
 '육십 사 년 초임검사로 임관한 김 전 실장은 십 년 후인 칠십 사 년 사 월 중앙정보부장 법률보좌관으로 파견됐고 같은 해 구 월 대공수사국장이 됐다',
 '하지만 이 같은 실수는 경쟁사를 의식해 졸속으로 진행되는 마트 할인 행사의 단면을 드러냈다는 점에서 소비자들의 시선이 곱지 않다',
 '김범석 쿠팡 대표는 

In [5]:
dataset['train'][0]

{'audio': {'path': None,
  'array': array([-3.05175781e-05,  0.00000000e+00, -3.05175781e-05, ...,
          0.00000000e+00,  0.00000000e+00, -6.10351562e-05]),
  'sampling_rate': 16000},
 'text': '인사를 결정하는 과정에서 당 지도부가 우 원내대표 및 원내지도부와 충분한 상의를 거치지 않은 채 일방적으로 인사를 했다는 불만도 원내지도부를 중심으로 흘러나왔다'}

In [6]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [7]:
input_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

In [8]:
input_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 457
    })
})

In [9]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [10]:
import evaluate
metric = evaluate.load('cer')

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [12]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    # return {"wer": wer}
    return {"cer": cer}


In [14]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-ko",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs = 2,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer", # 한국어의 경우 wer보다 cer이 나음
    greater_is_better=False,
    push_to_hub=False,  # True로 하면 huggingface에 push
)

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=input_dataset["train"],
    eval_dataset=input_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [16]:
trainer.evaluate(language="Korean")

{'eval_loss': 1.073493242263794,
 'eval_cer': 25.95006281407035,
 'eval_runtime': 83.7802,
 'eval_samples_per_second': 5.455,
 'eval_steps_per_second': 0.692}